In [1]:
!sudo apt-get update -y
!sudo apt-get install python3.9
!pip install --upgrade transformers
!pip install --upgrade simpletransformers
!pip install --upgrade rouge-metric

'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.


     ---------------------------------------- 5.8/5.8 MB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 182.4/182.4 kB ? eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.9.1
    Uninstalling huggingface-hub-0.9.1:
      Successfully uninstalled huggingface-hub-0.9.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.21.2
    Uninstalling transformers-4.21.2:
      Successfully uninstalled transformers-4.21.2


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: C:\Users\USER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t


     -------------------------------------- 250.5/250.5 kB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 8.7 MB/s eta 0:00:00
     ---------------------------------------- 43.6/43.6 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.9/1.9 MB 15.1 MB/s eta 0:00:00
     ---------------------------------------- 9.2/9.2 MB 22.6 MB/s eta 0:00:00
     ------------------------------------- 451.7/451.7 kB 27.6 MB/s eta 0:00:00
     ---------------------------------------- 182.5/182.5 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 526.5/526.5 kB 32.3 MB/s eta 0:00:00
     ------------------------------------- 169.0/169.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with sta

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: C:\Users\USER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
import math
import random
import time
import sys

from scipy import spatial
import numpy as np
from sklearn.utils import shuffle
import pandas as pd
# from transformers import T5Tokenizer, T5ForConditionalGeneration
from simpletransformers.language_representation import RepresentationModel
from simpletransformers.t5 import T5Model, T5Args

import torch

from rouge_metric import PyRouge
rouge = PyRouge()

In [ ]:
""" from google.colab import drive
drive.mount('/content/drive') """

Mounted at /content/drive


In [ ]:
""" os.chdir("/content/drive/MyDrive/Colab Notebooks")
print(os.getcwd()) """

/content/drive/MyDrive/Colab Notebooks


In [3]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# source_file = os.path.join(os.getcwd(), "graph-instruct.csv")
source_file = os.path.join(os.getcwd(), "./rephrase_402.csv")
print(source_file)
instruct = pd.read_csv(source_file)

c:\Users\USER\OneDrive\文件\intern\./rephrase_402.csv


In [ ]:
# source_file = os.path.join(os.getcwd(), "open_demo.csv")
# demo = pd.read_csv(source_file, usecols=['prompt', 'completion'])

In [5]:
instruct.summary[0]

'The classes of the nodes are: Actor Movie Film Director Genre  The list of relationships are: APPEARED_IN DIRECTED_BY BELONGS_TO  The properties of the classes: Movie: label title released actors director box_office Director: label gender(male/female) movie_directed nationality(UK US JP TW) Genre: label  Actor nodes are: Isabelle Huppert Sacueline Bissse Sandrine Bonnaire Bob Steele... Movie nodes are: Brief Encounter Casablanca Before Sunrise Breathless In the Mood for Love... Director nodes are: David Lean Michael Curtiz Richard Linklater Jean-Luc Godard Kar Wai Wong... Genre nodes are: Romance Crime Sci-Fi Horror...    The actions to interact with the canvas are: load remove select unselect  The method to perform the actions are: by class by relation by property by node by selected  the above methods are strictly the only 5 methods. The instructions are a set of commands which can display or remove nodes and relationships on a canvas.'

In [6]:
train = instruct.sample(frac = 0.7)
test = instruct.drop(train.index)

In [ ]:
# train = train.applymap(str)
# test = test.applymap(str)

In [9]:
train_df = pd.DataFrame()
#train_df['input_text'] = train['prompt']
train_df['input_text'] = train['summary'] + ", " + train['prompt']
train_df['input_text'] = "by class by relation by property by node by selected" + ", " + train['prompt']
#train_df['target_text'] = train['action'] + ", " + train['method']	+", " + train['relation']	+", " + train['clazz']	+", " + train['subject']	+", " + train['node']	+", " + train['property']
train['method'] = train['method'].str.replace(r'method: by ', '')
train['method'] = train['method'].str.replace(r'method: \'\'', 'None')
train_df['target_text'] = train['action']
train_df['target_text'] = train['method']
train_df["prefix"] = "summarize"
print("Number of training data: %d" % len(train_df))
eval_df = pd.DataFrame()
#eval_df['input_text'] = test['prompt']
eval_df['input_text'] = test['summary'] + ", " + test['prompt']
eval_df['input_text'] = "by class by relation by property by node by selected" + ", " + test['prompt']
#eval_df['target_text'] = test['action'] +", " + test['method']	+", " + test['relation']	+", " + test['clazz']	+", " + test['subject']	+", " + test['node']	+", " + test['property']
test['method'] = test['method'].str.replace(r'method: by ', '')
test['method'] = test['method'].str.replace(r'method: \'\'', 'None')
eval_df['target_text'] = test['action']
eval_df['target_text'] = test['method']
eval_df["prefix"] = "summarize"
print("Number of testing data: %d" % len(eval_df))

Number of training data: 281
Number of testing data: 120


C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.


In [10]:
for i in train['method']:
  print(i)

 property
 node
 relation
 class
 selected
 node
 relation
 class
 relation
 relation
 relation
 relation
 relation
 relation
 relation
 node
 node
 None
 property
 relation
 None
 property
 property
 node
 property
 property
 relation
 relations
 relation
 relation
 property
 property
 relation
 node
 class
 relation
 property
 node
 relation
 property
 class
 node
 relation
 relation
 property
 relations
 relation
 node
 node
 node
 class
 property
 None
 property
 relation
 property
 None
 relation
 relation
 property
 class
 node
 property
 relation
 relation
 class
 None
 relation
 relation
 property
 relation
 class
 relation
 class
 relation
 property
 property
 relation
 None
 class
 property
 relation
 relation
 relation
 node
 relation
 relation
 node
 relation
 class
 property
 node
 class
 class
 property
 relation
 relation
 class
 node
 relation
 property
 node
 property
 relation
 property
 property
 property
 relations
 relation
 property
 node
 class
 relation
 None
 n

In [11]:
class T5Summarizer:
    def __init__(self, max_seq_len=500, max_output_len=300, lang='en'):
        self.max_seq_len = max_seq_len
        self.max_output_len = max_output_len
        self.lang = lang
        self.model = None

    def load_default_model(self, model_path="google/mt5-base"):
        self.model = T5Model("mt5", model_path, use_cuda=True)

    def create_model(self, epoch=2):
        self.model_args = T5Args()
        self.model_args.num_beams = 2
        self.model_args.do_sample = True
        self.model_args.top_k = 3
        self.model_args.top_p = 0.95
        # self.model_args.learning_rate = 0.001
        self.model_args.preprocess_inputs = True
        self.model_args.max_seq_length = self.max_seq_len
        self.model_args.max_length = self.max_output_len
        self.model_args.train_batch_size = 2
        self.model_args.eval_batch_size = 8
        self.model_args.num_train_epochs = 30
        self.model_args.evaluate_during_training = False
        self.model_args.evaluate_during_training_steps = 300000
        self.model_args.use_multiprocessing = False
        self.model_args.fp16 = False
        self.model_args.save_steps = -1
        self.model_args.save_eval_checkpoints = False
        self.model_args.no_cache = True
        self.model_args.reprocess_input_data = True
        self.model_args.overwrite_output_dir = True # Please set to False if you don't want to overwirte your previous results
        self.model_args.num_return_sequences = 1
        self.model_args.output_dir = "outputs/"
        self.model = T5Model("mt5", "google/mt5-base", args=self.model_args, use_cuda=True)

    def load_data(self, data_type):
        #data_dir = "./CLTS"
        # with open ...
        #     for ...
        #         src = src.replace(...)[...]
        #         tgt = tgt.replace(...)[...]
        #         yield (src, tgt)
        ##################################################      
        # with open(os.path.join(data_dir, "%s.src" % data_type)) as fsrc, open(os.path.join(data_dir, "%s.tgt" % data_type)) as ftgt:
        #     for src, tgt in zip(fsrc, ftgt):
        #         src = src.replace(" ", "")[:self.max_seq_len]
        #         tgt = tgt.replace(" ", "")[:self.max_seq_len]
        #         yield (src, tgt)
        ##################################################
        return None

    def train(self, train_size=None, epoch=2):
        torch.cuda.empty_cache()
        torch.cuda.memory_summary(device=None, abbreviated=False)
        self.create_model(epoch=30)
        # for src, tgt in self.load_data("train"):
        #     train_data.append({"input_text": src, "target_text": tgt})
        #     if train_size and len(train_data) >= train_size:
        #         break
        self.model.train_model(train_df)

    def evaluate(self, valid_size=None):
        references = []
        predictions = []
        # for src, tgt in self.load_data("test"):
        #     eval_data.append("summarize: %s" % src)
        #     if self.lang == 'zh':
        #         tgt = " ".join(list(tgt))
        #     references.append([tgt])
        #     if valid_size and len(eval_data) >= valid_size:
        #        

        predicts = self.model.predict(eval_df['input_text'].values.tolist())
        predictions = [" ".join(list(p)) for p in predicts]
        # # if self.lang == 'zh':
        # #     predictions = [" ".join(list(p)) for p in predictions]
        #tgt = " ".join(list(eval_df['target_text']))
        references = eval_df['target_text'].values.tolist()
        print("Number of prediction data: %d" % len(predictions))
        print("Number of references data: %d" % len(references))
        
        return rouge.evaluate(predictions, references)
        #return self.model.evaluate(eval_df, output_dir = "./evaluate_result/")

    def perform(self, text, headline="", target_len=None):
        if not self.model:
            self.load_default_model()
        if not target_len:
            target_len = self.max_output_len
        result = self.model.predict(["summarize: %s" % text])[0]
        return result[:target_len]


In [12]:
model = T5Summarizer()

In [ ]:
#model.load_default_model("./outputs981")

In [15]:
# torch.cuda.init()

AssertionError: Torch not compiled with CUDA enabled

In [16]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [17]:
model.train()

KeyError: 'allocated_bytes.all.current'

In [ ]:
model.evaluate()

In [ ]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer
from transformers import pipeline

model = MT5ForConditionalGeneration.from_pretrained("./outputs")
tokenizer = T5Tokenizer.from_pretrained("google/mt5-large")

generator = pipeline(task="summarization", model=model, tokenizer=tokenizer)


In [ ]:
result = []
correct_count = 0
for i in eval_df['input_text'].index:
  ans = generator(eval_df['input_text'][i])
  result.append(ans)
  #print("Predict answer:", ans[0]['summary_text'], len(ans[0]['summary_text']), "  ", eval_df['target_text'][i], len(eval_df['target_text'][i]))
  if(ans[0]['summary_text'] == eval_df['target_text'][i].strip()):
    correct_count = correct_count + 1
  else:
      print( test['prompt'][i])
  

print(correct_count/len(eval_df['input_text']))


In [ ]:
eval_df['target_text']

In [ ]:
model.perform("The classes of the nodes are: Actor Movie Film Director Genre  The list of relationships are: APPEARED_IN DIRECTED_BY BELONGS_TO  The properties of the classes: Movie: label title released actors director box_office Director: label gender(male/female) movie_directed nationality(UK US JP TW) Genre: label  Actor nodes are: Isabelle Huppert Sacueline Bissse Sandrine Bonnaire Bob Steele... Movie nodes are: Brief Encounter Casablanca Before Sunrise Breathless In the Mood for Love... Director nodes are: David Lean Michael Curtiz Richard Linklater Jean-Luc Godard Kar Wai Wong... Genre nodes are: Romance Crime Sci-Fi Horror...    The actions to interact with the canvas are: load remove select unselect  The method to perform the actions are: by class by relation by property by node by selected  the above methods are strictly the only 5 methods. The instructions are a set of commands which can display or remove nodes and relationships on a canvas., show me the movie directed by Steven Spielberg")
#show me the movie directed by Steven Spielberg

In [ ]:
model.perform("The classes of the nodes are: Actor Movie Film Director Genre  The list of relationships are: APPEARED_IN DIRECTED_BY BELONGS_TO  The properties of the classes: Movie: label title released actors director box_office Director: label gender(male/female) movie_directed nationality(UK US JP TW) Genre: label  Actor nodes are: Isabelle Huppert Sacueline Bissse Sandrine Bonnaire Bob Steele... Movie nodes are: Brief Encounter Casablanca Before Sunrise Breathless In the Mood for Love... Director nodes are: David Lean Michael Curtiz Richard Linklater Jean-Luc Godard Kar Wai Wong... Genre nodes are: Romance Crime Sci-Fi Horror...    The actions to interact with the canvas are: load remove select unselect  The method to perform the actions are: by class by relation by property by node by selected  the above methods are strictly the only 5 methods. The instructions are a set of commands which can display or remove nodes and relationships on a canvas., Show me all female actors")
#Show me all female actors

In [ ]:
model.perform("The classes of the nodes are: Actor Movie Film Director Genre  The list of relationships are: APPEARED_IN DIRECTED_BY BELONGS_TO  The properties of the classes: Movie: label title released actors director box_office Director: label gender(male/female) movie_directed nationality(UK US JP TW) Genre: label  Actor nodes are: Isabelle Huppert Sacueline Bissse Sandrine Bonnaire Bob Steele... Movie nodes are: Brief Encounter Casablanca Before Sunrise Breathless In the Mood for Love... Director nodes are: David Lean Michael Curtiz Richard Linklater Jean-Luc Godard Kar Wai Wong... Genre nodes are: Romance Crime Sci-Fi Horror...    The actions to interact with the canvas are: load remove select unselect  The method to perform the actions are: by class by relation by property by node by selected  the above methods are strictly the only 5 methods. The instructions are a set of commands which can display or remove nodes and relationships on a canvas., Show actors")
#Show actors

In [ ]:
model.perform("The classes of the nodes are: Actor Movie Film Director Genre  The list of relationships are: APPEARED_IN DIRECTED_BY BELONGS_TO  The properties of the classes: Movie: label title released actors director box_office Director: label gender(male/female) movie_directed nationality(UK US JP TW) Genre: label  Actor nodes are: Isabelle Huppert Sacueline Bissse Sandrine Bonnaire Bob Steele... Movie nodes are: Brief Encounter Casablanca Before Sunrise Breathless In the Mood for Love... Director nodes are: David Lean Michael Curtiz Richard Linklater Jean-Luc Godard Kar Wai Wong... Genre nodes are: Romance Crime Sci-Fi Horror...    The actions to interact with the canvas are: load remove select unselect  The method to perform the actions are: by class by relation by property by node by selected  the above methods are strictly the only 5 methods. The instructions are a set of commands which can display or remove nodes and relationships on a canvas., Actors who appeared in Casablanca")
#Actors who appeared in Casablanca

In [ ]:
model.perform("The classes of the nodes are: Actor Movie Film Director Genre  The list of relationships are: APPEARED_IN DIRECTED_BY BELONGS_TO  The properties of the classes: Movie: label title released actors director box_office Director: label gender(male/female) movie_directed nationality(UK US JP TW) Genre: label  Actor nodes are: Isabelle Huppert Sacueline Bissse Sandrine Bonnaire Bob Steele... Movie nodes are: Brief Encounter Casablanca Before Sunrise Breathless In the Mood for Love... Director nodes are: David Lean Michael Curtiz Richard Linklater Jean-Luc Godard Kar Wai Wong... Genre nodes are: Romance Crime Sci-Fi Horror...    The actions to interact with the canvas are: load remove select unselect  The method to perform the actions are: by class by relation by property by node by selected  the above methods are strictly the only 5 methods. The instructions are a set of commands which can display or remove nodes and relationships on a canvas., Show me all female director")
#Show me all female director

In [ ]:
model.perform("The classes of the nodes are: Actor Movie Film Director Genre  The list of relationships are: APPEARED_IN DIRECTED_BY BELONGS_TO  The properties of the classes: Movie: label title released actors director box_office Director: label gender(male/female) movie_directed nationality(UK US JP TW) Genre: label  Actor nodes are: Isabelle Huppert Sacueline Bissse Sandrine Bonnaire Bob Steele... Movie nodes are: Brief Encounter Casablanca Before Sunrise Breathless In the Mood for Love... Director nodes are: David Lean Michael Curtiz Richard Linklater Jean-Luc Godard Kar Wai Wong... Genre nodes are: Romance Crime Sci-Fi Horror...    The actions to interact with the canvas are: load remove select unselect  The method to perform the actions are: by class by relation by property by node by selected  the above methods are strictly the only 5 methods. The instructions are a set of commands which can display or remove nodes and relationships on a canvas., Drop the movies with the box office more than 1 million")
#Drop the movies with the box office more than 1 million

In [ ]:
model.perform("The classes of the nodes are: Actor Movie Film Director Genre  The list of relationships are: APPEARED_IN DIRECTED_BY BELONGS_TO  The properties of the classes: Movie: label title released actors director box_office Director: label gender(male/female) movie_directed nationality(UK US JP TW) Genre: label  Actor nodes are: Isabelle Huppert Sacueline Bissse Sandrine Bonnaire Bob Steele... Movie nodes are: Brief Encounter Casablanca Before Sunrise Breathless In the Mood for Love... Director nodes are: David Lean Michael Curtiz Richard Linklater Jean-Luc Godard Kar Wai Wong... Genre nodes are: Romance Crime Sci-Fi Horror...    The actions to interact with the canvas are: load remove select unselect  The method to perform the actions are: by class by relation by property by node by selected  the above methods are strictly the only 5 methods. The instructions are a set of commands which can display or remove nodes and relationships on a canvas., Movies that start with 'j'")
#Movies that start with 'j'